In [136]:
import numpy as np
import tensorflow as tf
import re

##### 
    ADJ: adjective
    ADP: adposition
    ADV: adverb
    AUX: auxiliary
    CCONJ: coordinating conjunction
    DET: determiner
    INTJ: interjection
    NOUN: noun
    NUM: numeral
    PART: particle
    PRON: pronoun
    PROPN: proper noun
    PUNCT: punctuation
    SCONJ: subordinating conjunction
    SYM: symbol
    VERB: verb
    X: other


In [137]:
line ="ADJ ADP ADV AUX CCONJ DET INTJ NOUN NUM PART PRON PROPN PUNCT SCONJ SYM VERB X"
tags_list = line.split(' ')
line = "A B C D E F G H I J K L M N O P Q R S T U V W X Y Z"
letters = line.split(' ')

In [155]:
#extract words and embeddings from fasttext's output
skip = False
words = []
vecs = []
with open("/home/boeing/fasttext150", "r") as f:
    for line in f:
        if skip:
            info = line.split(' ')
            words.append(info[0]) 
            c = []
            for i in range(1, len(info)-1):
                c.append(float(info[i]))
            vecs.append(c)
        else:
            skip = True
            
with open('words', 'w') as f:
    for word in words:
        f.write(word+"\n")

npvecs = np.asarray(vecs)
npvecs.dump('vecs')

In [109]:
#create words(text8), tags, suf, pref dictionaries. id = 0 is reserved
word_id = {}
tag_id = {}
pref_id = {}
#suf_id = {} - same as pref_id
_id = 1
counter = 0
for word in words:
    if word not in word_id:
        word_id[word] = [_id, vecs[counter]]
        _id += 1
    counter += 1
word_id["%NOWORD%"] = [0, np.zeros(100)] #NOWORD is for no word. id = 0. embedding = 0 0 0 0 0 0..

_id = 1
for tag in tags_list:
    tag_id[tag] = _id
    _id += 1

_id = 1
for letter1 in letters:
    for letter2 in letters:
        pref_id[letter1.lower()+letter2.lower()]=_id
        _id +=1

In [56]:
#haha benis
print(word_id['penis'])

9752


In [43]:
#extract words and their tags from file from universaltransitions.com
tag_words = []
tag_tags = []
with open("/home/boeing/en_pud-ud-test.conllu", "r") as f:
    for line in f:
        if line[0] == '#':
            continue
        splitted = line.split('\t')
        if len(splitted) <= 3:
            continue
        tag_words.append(splitted[1])
        tag_tags.append(splitted[3])

with open('tag_words', 'w') as f:
    for word in tag_words:
        f.write(word+"\n")
        
with open('tag_tags', 'w') as f:
    for tag in tag_tags:
        f.write(tag+"\n")

In [157]:
#let's find out how many words have no embs.
words_out_of_dict = []
with open("words_out_of_dict", "w") as f:
    for word in tag_words:
        if word not in ['.',',',':','"', '\'', '”', '“']:
            if word.lower() not in words:
                if word.lower() not in words_out_of_dict:
                    f.write(word+"\n")
                    words_out_of_dict.append(word)

In [158]:
uniq = []
for word in tag_words:
    if word not in ['.',',',':','"', '\'', '”', '“']:
        if word not in uniq:
            uniq.append(word)
print(len(uniq))
print(len(words_out_of_dict))

5727
674


In [77]:
openedvecs = np.load('vecs')
#print(openedvecs[0])

In [159]:
suf_init = tf.zeros(shape=[len(pref_id)+1,10], dtype="float32")
suf = tf.Variable(suf_init, name='suffixes') #нулевая по счету строка - нулевая. 
pref = tf.Variable(suf_init, name='prefixes') #нулевая по счету строка - нулевая.
print(suf)

<tf.Variable 'suffixes_3:0' shape=(677, 10) dtype=float32_ref>


In [161]:
tag_init = tf.zeros(shape=[len(tag_id)+1,100], dtype = "float32")
tags = tf.Variable(tag_init, name='tags') #нулевая по счету строка - нулевая.
print(tags)

<tf.Variable 'tags_2:0' shape=(18, 100) dtype=float32_ref>


In [131]:
#когда будем присваивать слову тэг, будем создавать словарь tags_learned "номер слова в выборке" - "tag_id".
#тогда одному и тому же слову можно будет присваивать разные тэги! tag_id потом будет цепляться к вариаблу.

def suf(word):
    return word[-2]+word[-1]
def pref(word):
    return word[0]+word[1]
#ОЧЕНЬ БЫДЛОКОД НО ЗАТО ПОНЯТНО И ВИДНО ЧТО ПРОИСХОДИТ В СУПЕРВЕКТОРЕ
def supervector_maker(words, tags_learned, i):
    if not (0<=i<=len(words)-1):
        print("i out of range")
        return 0
    
    if i>2 and i<len(words)-3:

        c = np.array(word_id[words[i-3]][0], tags_learned[i-3], pref_id[suf(words[i-3])],\
                     pref_id[pref(words[i-3])], \
                     word_id[words[i-2]][0], tags_learned[i-2], pref_id[suf(words[i-2])],\
                     pref_id[pref(words[i-2])], \
                     word_id[words[i-1]][0], tags_learned[i-1], pref_id[suf(words[i-1])],\
                     pref_id[pref(words[i-1])], \
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     word_id[words[i+1]][0], pref_id[suf(words[i+1])],\
                     pref_id[pref(words[i+1])], \
                     word_id[words[i+2]][0], pref_id[suf(words[i+2])],\
                     pref_id[pref(words[i+2])], \
                     word_id[words[i+3]][0], pref_id[suf(words[i+3])],\
                     pref_id[pref(words[i+3])])
    if i==0:
        c = np.array(0, 0, 0, 0,\
                     0, 0, 0, 0,\
                     0, 0, 0, 0,\
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     word_id[words[i+1]][0], pref_id[suf(words[i+1])],\
                     pref_id[pref(words[i+1])], \
                     word_id[words[i+2]][0], pref_id[suf(words[i+2])],\
                     pref_id[pref(words[i+2])], \
                     word_id[words[i+3]][0], pref_id[suf(words[i+3])],\
                     pref_id[pref(words[i+3])])
    if i==1:
        c = np.array(0, 0, 0, 0,\
                     0, 0, 0, 0,\
                     word_id[words[i-1]][0], tags_learned[i-1], pref_id[suf(words[i-1])],\
                     pref_id[pref(words[i-1])], \
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     word_id[words[i+1]][0], pref_id[suf(words[i+1])],\
                     pref_id[pref(words[i+1])], \
                     word_id[words[i+2]][0], pref_id[suf(words[i+2])],\
                     pref_id[pref(words[i+2])], \
                     word_id[words[i+3]][0], pref_id[suf(words[i+3])],\
                     pref_id[pref(words[i+3])])
    if i==2:
        c = np.array(0, 0, 0, 0,\
                     word_id[words[i-2]][0], tags_learned[i-2], pref_id[suf(words[i-2])],\
                     pref_id[pref(words[i-2])], \
                     word_id[words[i-1]][0], tags_learned[i-1], pref_id[suf(words[i-1])],\
                     pref_id[pref(words[i-1])], \
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     word_id[words[i+1]][0], pref_id[suf(words[i+1])],\
                     pref_id[pref(words[i+1])], \
                     word_id[words[i+2]][0], pref_id[suf(words[i+2])],\
                     pref_id[pref(words[i+2])], \
                     word_id[words[i+3]][0], pref_id[suf(words[i+3])],\
                     pref_id[pref(words[i+3])])
    if i==len(words)-1:
        c = np.array(word_id[words[i-3]][0], tags_learned[i-3], pref_id[suf(words[i-3])],\
                     pref_id[pref(words[i-3])], \
                     word_id[words[i-2]][0], tags_learned[i-2], pref_id[suf(words[i-2])],\
                     pref_id[pref(words[i-2])], \
                     word_id[words[i-1]][0], tags_learned[i-1], pref_id[suf(words[i-1])],\
                     pref_id[pref(words[i-1])], \
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     0, 0, 0, \
                     0, 0, 0, \
                     0, 0, 0)
    if i==len(words)-2:
        c = np.array(word_id[words[i-3]][0], tags_learned[i-3], pref_id[suf(words[i-3])],\
                     pref_id[pref(words[i-3])], \
                     word_id[words[i-2]][0], tags_learned[i-2], pref_id[suf(words[i-2])],\
                     pref_id[pref(words[i-2])], \
                     word_id[words[i-1]][0], tags_learned[i-1], pref_id[suf(words[i-1])],\
                     pref_id[pref(words[i-1])], \
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     word_id[words[i+1]][0], pref_id[suf(words[i+1])],\
                     pref_id[pref(words[i+1])], \
                     0, 0, 0, \
                     0, 0, 0)
    if i==len(words)-3:
        c = np.array(word_id[words[i-3]][0], tags_learned[i-3], pref_id[suf(words[i-3])],\
                     pref_id[pref(words[i-3])], \
                     word_id[words[i-2]][0], tags_learned[i-2], pref_id[suf(words[i-2])],\
                     pref_id[pref(words[i-2])], \
                     word_id[words[i-1]][0], tags_learned[i-1], pref_id[suf(words[i-1])],\
                     pref_id[pref(words[i-1])], \
                     word_id[words[i]][0], pref_id[suf(words[i])],\
                     pref_id[pref(words[i])], \
                     word_id[words[i+1]][0], pref_id[suf(words[i+1])],\
                     pref_id[pref(words[i+1])], \
                     word_id[words[i+2]][0], pref_id[suf(words[i+2])],\
                     pref_id[pref(words[i+2])], \
                     0, 0, 0)
    return c

In [81]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 1140 # SUPERVECTOR: 3*4*220 + 4*3*120
num_classes = 17 # 17 classes, but there are 18 tag id's.


# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, 17])) #softmax here/17 classes
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([17]))
}

In [ ]:
X = tf.placeholder("float", [None, num_input]) #batches
Y = tf.placeholder("int", [None, num_classes]) #Ys: id of tags

In [ ]:
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [ ]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

